In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df

,Trader,Asset Name,Status,Trade Type,Payout,Investment,Trader Profit,Trading Strike,Close Strike,Trading Time,Close Time
0,trader C,AUD/USD,Closed,Put,1.85,1000,-1000.000,0.69676,0.69724,2020-06-10 03:00:01.760,2020-06-10 03:05:00.000
1,trader C,EUR/JPY,Closed,Call,1.85,10000,8500.000,116.35500,116.36400,2020-04-27 02:30:01.370,2020-04-27 02:35:00.000
2,trader C,EUR/USD,Closed,Call,1.85,5000,-5000.000,1.09412,1.09378,2020-04-15 07:45:01.810,2020-04-15 07:50:00.000
3,trader C,EUR/JPY,Closed,Call,1.95,1000,950.000,119.69900,119.71600,2019-09-13 18:57:25.390,2019-09-13 18:58:25.390
4,trader C,EUR/USD,Closed,Put,1.85,1000,-1000.000,1.21202,1.21202,2020-12-03 02:55:00.520,2020-12-03 03:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...
3394,trader C,AUD/USD,Closed,Call,1.85,1000,-1000.000,0.76130,0.76091,2021-04-04 23:00:58.950,2021-04-04 23:05:58.950
3395,trader C,EUR/USD,Closed,Call,1.88,20000,17600.000,1.12285,1.12358,2020-03-06 05:59:58.740,2020-03-06 06:05:00.000
3396,trader C,NZD/JPY,Closed,Put,1.90,1000,-1000.000,63.70600,63.71300,2020-04-02 01:28:05.140,2020-04-02 01:31:05.140
3397,trader A,BITCOIN,Sold,Call,2.05,3435,2662.041,32669.30000,32829.60000,2021-06-21 06:37:12.820,2021-06-21 06:38:10.123


In [5]:
df = df.drop(['Asset Name', 'Trading Time', 'Close Time'], axis=1)

In [6]:
df['Status'].replace(['Closed', 'Sold'], [0,1], inplace=True)
df['Trade Type'].replace(['Call', 'Put'], [0,1], inplace=True)

In [7]:
df

,Trader,Status,Trade Type,Payout,Investment,Trader Profit,Trading Strike,Close Strike
0,trader C,0,1,1.85,1000,-1000.000,0.69676,0.69724
1,trader C,0,0,1.85,10000,8500.000,116.35500,116.36400
2,trader C,0,0,1.85,5000,-5000.000,1.09412,1.09378
3,trader C,0,0,1.95,1000,950.000,119.69900,119.71600
4,trader C,0,1,1.85,1000,-1000.000,1.21202,1.21202
...,...,...,...,...,...,...,...,...
3394,trader C,0,0,1.85,1000,-1000.000,0.76130,0.76091
3395,trader C,0,0,1.88,20000,17600.000,1.12285,1.12358
3396,trader C,0,1,1.90,1000,-1000.000,63.70600,63.71300
3397,trader A,1,0,2.05,3435,2662.041,32669.30000,32829.60000


In [8]:
dfa = df[df['Trader'] == 'trader A']
dfb = df[df['Trader'] == 'trader B']
dfc = df[df['Trader'] == 'trader C']
dfe = df[df['Trader'] == 'trader E']

In [9]:
dfa = dfa.drop(['Trader'], axis=1)
dfb = dfb.drop(['Trader'], axis=1)
dfc = dfc.drop(['Trader'], axis=1)
dfe = dfe.drop(['Trader'], axis=1)

In [10]:
dfa

,Status,Trade Type,Payout,Investment,Trader Profit,Trading Strike,Close Strike
5,0,1,2.05,4549,4776.450,37689.7,37671.8
170,0,1,2.05,5594,5873.700,37911.5,37863.8
211,0,1,2.05,5590,-5590.000,37348.2,37491.6
244,0,1,2.05,5625,-5625.000,37703.4,37774.8
318,0,1,2.05,5031,-5031.000,37588.7,37651.3
...,...,...,...,...,...,...,...
3344,0,0,2.05,7368,-7368.000,34722.8,34674.5
3382,0,0,2.05,3679,3862.950,37727.9,37767.8
3384,0,0,2.05,6083,-6083.000,37508.5,37347.8
3391,0,0,2.30,2000,-2000.000,36291.9,36259.8


In [11]:
from sklearn.ensemble import IsolationForest

In [12]:
datacopy = dfa.copy()
datacopy.columns
to_model_cols = datacopy.columns[4:8]
clf = IsolationForest(n_estimators=10, max_samples='auto', contamination=float(.0009), max_features=1.0, bootstrap=False, n_jobs=-1, random_state=12345, verbose=0)
clf.fit(datacopy[to_model_cols])

pred = clf.predict(datacopy[to_model_cols])

datacopy['Class'] = pred
datacopy['Class'].value_counts()

 1    99
-1     1
Name: Class, dtype: int64

In [13]:
datacopy = dfb.copy()

clf = IsolationForest(n_estimators=10, max_samples='auto', contamination=float(.0009), max_features=1.0, bootstrap=False, n_jobs=-1, random_state=12345, verbose=0)
clf.fit(datacopy)

pred = clf.predict(datacopy)

datacopy['Class'] = pred
datacopy['Class'].value_counts()

 1    28
-1     1
Name: Class, dtype: int64

In [14]:
datacopy = dfc.copy()

clf = IsolationForest(n_estimators=10, max_samples='auto', contamination=float(.0009), max_features=1.0, bootstrap=False, n_jobs=-1, random_state=12345, verbose=0)
clf.fit(datacopy)

pred = clf.predict(datacopy)

datacopy['Class'] = pred
datacopy['Class'].value_counts()

 1    3239
-1       3
Name: Class, dtype: int64

In [15]:
datacopy = dfe.copy()

clf = IsolationForest(n_estimators=10, max_samples='auto', contamination=float(.0009), max_features=1.0, bootstrap=False, n_jobs=-1, random_state=12345, verbose=0)
clf.fit(datacopy)

pred = clf.predict(datacopy)

datacopy['Class'] = pred
datacopy['Class'].value_counts()

 1    27
-1     1
Name: Class, dtype: int64

In [16]:
df = df.drop(['Trader'], axis=1)

In [17]:
datacopy = df.copy()
to_model_cols = datacopy.columns[4:8]

clf = IsolationForest(n_estimators=10, max_samples='auto', contamination=float(.0009), max_features=1.0, bootstrap=False, n_jobs=-1, random_state=12345, verbose=0)
clf.fit(datacopy[to_model_cols])

pred = clf.predict(datacopy[to_model_cols])

datacopy['Class'] = pred
datacopy['Class'].value_counts()

 1    3396
-1       3
Name: Class, dtype: int64